In [1]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json

def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")

def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())

def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())

def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())

functions_map = {
    "get_ticker":get_ticker,
    "get_income_statement":get_income_statement,
    "get_balance_sheet":get_balance_sheet,
    "get_daily_stock_performance":get_daily_stock_performance
}

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

In [2]:
import openai as client

# 처음 assistant를 생성할때는 아래 코드를 이용하면됨
# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#     model="gpt-3.5-turbo-1106",
#     tools=functions
# ) # 2024.03.07 현재 assistant가 beta임

# 한번 생성한 assistant를 재사용할때는 id를 이용해서 사용가능
assistant_id="asst_xxxxxxx"

In [ ]:
# thread를 생성 (thread는 메시지의 그룹임). 생성한 thread는 run을 해야 llm에서 처리됨
thread = client.beta.threads.create(
    messages=[
        {
            "role":"user",
            "content":"I want to know if the Salesforce stock is a good buy"
        }
    ]
)
thread

In [ ]:
# thread를 실행하기 위해 run을 생성
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id
)
run

In [37]:
import json

# 현재(2024.3월) streaming이 지원되지 않으므로 run의 상태가 어떤지 알기 위해서 계속 물어봐야함
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )

def get_messages(thread_id):
    messages = client.beta.threads.messages.list(
        thread_id=thread_id
    )
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")

def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=content
    )

def cancel_run(run_id, thread_id):
    return client.beta.threads.runs.cancel(
        thread_id=thread_id,
        run_id=run_id
    )

def delete_thread(thread_id):
    return client.beta.threads.delete(
        thread_id=thread_id
    )

def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        # ai가 실행하라고 한 함수를 실행. 함수는 미리 functions_map에 담아놨음
        outputs.append({
            "output": functions_map[function.name](json.loads(function.arguments)),
            "tool_call_id":action_id
        })

    return outputs

def submit_tool_outputs(run_id, thread_id):
    outputs = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id = run_id,
        thread_id=thread_id,
        tool_outputs=outputs
    )

In [48]:
get_run(run.id, thread.id).status

'completed'

In [49]:
get_messages(thread.id)

user: I want to know if the Salesforce stock is a good buy
assistant: The ticker symbol for Salesforce is CRM. Here are some key financial metrics for Salesforce:

- Net Income: $4.13 Billion
- Earnings Per Share (EPS): $4.25
- Total Revenue: $34.857 Billion
- Gross Profit: $26.316 Billion
- Operating Income: $5.999 Billion
- EBITDA: $9.958 Billion

The stock performance for Salesforce over the last 100 days has been as follows:

- Highest Stock Price: $318.71
- Lowest Stock Price: $249.84
- Current Stock Price: $306.00

Based on this information, we can see that Salesforce has consistent profitability, a strong gross profit margin, and a healthy EBITDA. The stock price has also shown significant growth over the past 100 days.

Given these positive indicators, Salesforce stock appears to be a good buy, but it's important to consider your investment strategy and consult with a financial advisor before making any investment decisions.
user: Please go ahead


In [ ]:
send_message(thread.id, "Please go ahead")

In [ ]:
submit_tool_outputs(run.id, thread.id)